# Python Project for Data Science
Module 1
Reading: yfinance Librar

https://www.coursera.org/learn/python-project-for-data-science/ungradedWidget/kNKHN/reading-yfinance-library


In [4]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import yfinance as yf
# Download historical data for a stock
msft = yf.Ticker("MSFT")
msft_data = msft.history(period="max")
# Display the downloaded data
msft_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.054376,0.062373,0.054376,0.059707,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.059707,0.062906,0.059707,0.061839,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.061839,0.063439,0.061839,0.062906,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.062906,0.063439,0.060773,0.061306,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.061306,0.061839,0.059707,0.060240,47894400,0.0,0.0


In [6]:
import yfinance as yf
# Download historical data for a stock
msft = yf.Ticker("AMD")
msft_data = msft.history(period="max")
# Display the downloaded data
msft_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-03-17 00:00:00-05:00,0.0,3.302083,3.125000,3.145833,219600,0.0,0.0
1980-03-18 00:00:00-05:00,0.0,3.125000,2.937500,3.031250,727200,0.0,0.0
1980-03-19 00:00:00-05:00,0.0,3.083333,3.020833,3.041667,295200,0.0,0.0
1980-03-20 00:00:00-05:00,0.0,3.062500,3.010417,3.010417,159600,0.0,0.0
1980-03-21 00:00:00-05:00,0.0,3.020833,2.906250,2.916667,130800,0.0,0.0


In [8]:
# Robust price downloader v2: try yfinance; on 429 fall back to Stooq CSV (tsla.us / gme.us).
# Pandas 2.x 対応（io.StringIO 使用）。

import io, time, random, datetime as dt
import pandas as pd, yfinance as yf, requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def _make_session(total=2, backoff_factor=0.7) -> requests.Session:
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/125.0.0.0 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9,ja;q=0.8",
        "Connection": "keep-alive",
        "DNT": "1",
        "Cache-Control": "no-cache",
    })
    retry = Retry(
        total=total, read=total, connect=total,
        backoff_factor=backoff_factor,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=["HEAD","GET","OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=20)
    s.mount("https://", adapter); s.mount("http://", adapter)
    return s

SESSION = _make_session()

def _stooq_symbol(symbol: str) -> str:
    return f"{symbol.lower()}.us"

def _read_stooq_csv(symbol: str, start: str) -> pd.DataFrame:
    # Stooq 直CSV。例: https://stooq.com/q/d/l/?s=tsla.us&i=d
    url = f"https://stooq.com/q/d/l/?s={_stooq_symbol(symbol)}&i=d"
    r = SESSION.get(url, timeout=30)
    r.raise_for_status()
    df = pd.read_csv(io.StringIO(r.text))  # Pandas 2.x OK
    # 列: Date,Open,High,Low,Close,Volume
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"]).sort_values("Date").set_index("Date")
    if start:
        df = df.loc[df.index >= pd.to_datetime(start)]
    # yfinance に近い列順へ（Adj Close はない）
    cols = [c for c in ["Open","High","Low","Close","Volume"] if c in df.columns]
    df = df[cols].copy()
    df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce").fillna(0).astype("int64")
    return df

def _sleep_backoff(attempt, base=1.4, max_wait=8.0):
    import math, random, time
    wait = min(max_wait, (base ** attempt) + random.uniform(0, 0.9))
    time.sleep(wait)

def robust_prices(symbol: str, start="2015-01-01", end=None, interval="1d") -> pd.DataFrame:
    """
    1) yfinance を軽く1回だけ試す（週足ウォームアップ→日足）
    2) 取得できなければ Stooq CSV にフォールバック
    """
    if end is None:
        end = (dt.date.today() + dt.timedelta(days=1)).isoformat()

    # 軽量ウォームアップ（429緩和用）— 1回だけ
    try:
        _ = yf.download(
            symbol, start=start, end=end, interval="1wk",
            progress=False, threads=False, prepost=False,
            repair=True, session=SESSION, timeout=20
        )
    except Exception:
        pass  # 失敗しても続行

    # 本命（日足）— 1〜2回だけ（連打しない）
    last_e = None
    for attempt in range(1, 3):
        try:
            df = yf.download(
                symbol, start=start, end=end, interval=interval,
                auto_adjust=False, progress=False, prepost=False,
                repair=True, threads=False, session=SESSION, timeout=30
            )
            if isinstance(df.index, pd.DatetimeIndex) and not df.empty:
                cols = [c for c in ["Open","High","Low","Close","Adj Close","Volume"] if c in df.columns]
                return df[cols]
        except Exception as e:
            last_e = e
        _sleep_backoff(attempt)

    # --- フォールバック：Stooq ---
    stq = _read_stooq_csv(symbol, start=start)
    # yfinance 互換に近づける（Adj Close なし）
    return stq[["Open","High","Low","Close","Volume"]]


requests-cache で同じURLの再叩きを回避

同意ページ（fc.yahoo.com）へ先にアクセスしてクッキーをセッションに載せる

指数バックオフ＋ジッターで429を回避

リクエスト最小化（期間短縮・週次に落としてから必要に応じて日次へ）

threads=False / repair=True / session=SESSION を徹底

In [9]:
# Q1: TSLA の終値と出来高
tsla_hist_all = robust_prices("TSLA", start="2015-01-01", interval="1d")
tsla_hist = tsla_hist_all.loc[:, [c for c in tsla_hist_all.columns if c in ["Close","Volume"]]].copy()
tsla_hist.tail()


YF.download() has changed argument auto_adjust default to True



1 Failed download:
['TSLA']: RetryError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by ResponseError('too many 429 error responses'))"))

1 Failed download:
['TSLA']: RetryError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by ResponseError('too many 429 error responses'))"))

1 Failed download:
['TSLA']: RetryError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by ResponseError('too many 429 error responses'))"))


,Close,Volume
Date,,
2025-08-08,329.65,91200319
2025-08-11,339.03,105320174
2025-08-12,340.84,80690111
2025-08-13,339.38,67838892
2025-08-14,335.58,74795974


In [17]:

tesla = yf.Ticker("TSLA")

In [18]:
tesla_data = tesla.history(period="max")

$TSLA: possibly delisted; no price data found  (1d 1926-09-08 -> 2025-08-14)


In [19]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume


In [12]:
# Q2/Q4 兼用：SECのCompany Facts APIから四半期Revenueを取得（アンチボット対応）
# 依存: requests, pandas
# 既存の `session`（make_hardened_session() などで作ったもの）がある前提

import requests, pandas as pd
from typing import List, Optional

# CIKは10桁ゼロパディング
def _cik(s: str|int) -> str:
    s = str(s).lstrip("0")
    return f"{int(s):010d}"

# SECは連絡先入りUser-Agent必須
def _sec_session(base_session: Optional[requests.Session] = None,
                 ua: str = "AssignmentNotebook/1.0 (contact@self-renova.com)") -> requests.Session:
    s = requests.Session()
    if base_session is not None:
        # 既存のヘッダーを継承（クッキー等は不要）
        s.headers.update(base_session.headers)
    s.headers.update({
        "User-Agent": ua,
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9,ja;q=0.8",
        "Connection": "keep-alive",
        "Cache-Control": "no-cache",
    })
    return s

# SECのcompanyfactsから四半期Revenueを抽出
def get_quarterly_revenue_from_sec(cik: str|int, base_session: Optional[requests.Session]=None) -> pd.DataFrame:
    cik10 = _cik(cik)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik10}.json"
    s = _sec_session(base_session)

    r = s.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()

    facts = data.get("facts", {}).get("us-gaap", {})
    # 候補となるRevenue科目（上から優先）
    candidates: List[str] = [
        "RevenueFromContractWithCustomerExcludingAssessedTax",
        "Revenues",
        "SalesRevenueNet",
        "SalesRevenueGoodsNet",
        "SalesRevenueServicesNet",
        "Revenue",
    ]

    found = None
    units_key = None
    for key in candidates:
        if key in facts and "units" in facts[key]:
            # USD優先。なければ最初のユニットを使う
            units = facts[key]["units"]
            if "USD" in units:
                found, units_key = key, "USD"
                break
            elif len(units) > 0:
                found, units_key = key, list(units.keys())[0]
                break

    if not found:
        raise RuntimeError("No revenue concept found in SEC companyfacts.")

    entries = facts[found]["units"][units_key]

    # 四半期に限定：fp ∈ {Q1, Q2, Q3, Q4} のものを採用
    rows = []
    for e in entries:
        fp = e.get("fp")
        if fp not in {"Q1", "Q2", "Q3", "Q4"}:
            continue  # 年次FYは除外
        val = e.get("val")
        end = e.get("end")
        if val is None or end is None:
            continue
        rows.append({"Date": pd.to_datetime(end, errors="coerce"), "Revenue": pd.to_numeric(val, errors="coerce")})

    df = pd.DataFrame(rows).dropna()
    if df.empty:
        raise RuntimeError("SEC returned no quarterly revenue rows.")

    # 同一Dateが複数ある場合は最新を採用
    df = df.sort_values("Date").drop_duplicates(subset=["Date"], keep="last").reset_index(drop=True)

    # 最終整形：Date昇順、必要列のみ
    df = df.sort_values("Date")[["Date", "Revenue"]].reset_index(drop=True)
    return df

# --- Q2: Tesla Revenue（CIK: 0001318605） ---
try:
    tsla_revenue = get_quarterly_revenue_from_sec("0001318605", base_session=session)
    print(tsla_revenue.tail())
except Exception as e:
    raise RuntimeError(f"Failed to fetch TSLA revenue via SEC API: {e}")

# --- Q4: GameStop Revenue（CIK: 0001326380） ---
try:
    gme_revenue = get_quarterly_revenue_from_sec("0001326380", base_session=session)
    print(gme_revenue.tail())
except Exception as e:
    raise RuntimeError(f"Failed to fetch GME revenue via SEC API: {e}")


         Date      Revenue
7  2024-03-31  21301000000
8  2024-06-30  25500000000
9  2024-09-30  25182000000
10 2025-03-31  19335000000
11 2025-06-30  22496000000
         Date     Revenue
12 2021-05-01  1276800000
13 2021-07-31  1183400000
14 2021-10-30  1296600000
15 2024-05-04   881800000
16 2025-05-03   732400000


In [2]:
import yfinance as yf

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.

In [20]:

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data  = requests.get(url).text

In [21]:

soup = BeautifulSoup(html_data,"html5lib")

In [22]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):

    if ('Tesla Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')
        
        for row in rows:
            col = row.find_all('td')
            
            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

In [23]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")

<>:1: SyntaxWarning:

invalid escape sequence '\$'

<>:1: SyntaxWarning:

invalid escape sequence '\$'

C:\Users\Mi PC\AppData\Local\Temp\ipykernel_33016\349343550.py:1: SyntaxWarning:

invalid escape sequence '\$'



In [ ]:
#Q3  GME の終値と出来高（日足）
gme_hist_all = robust_prices("GME", start="2010-01-01", interval="1d")
gme_hist = gme_hist_all.loc[:, [c for c in ["Close","Volume"] if c in gme_hist_all.columns]].copy()
gme_hist.tail()



1 Failed download:
['GME']: RetryError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by ResponseError('too many 429 error responses'))"))

1 Failed download:
['GME']: RetryError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by ResponseError('too many 429 error responses'))"))

1 Failed download:
['GME']: RetryError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by ResponseError('too many 429 error responses'))"))


,Close,Volume
Date,,
2025-08-08,22.27,4833088
2025-08-11,22.41,5212532
2025-08-12,22.93,4887648
2025-08-13,23.02,5349328
2025-08-14,22.89,4759618


In [13]:
# Q4: GameStop Revenue（SEC公式API利用）
# すでに get_quarterly_revenue_from_sec 関数はQ2で定義済みなので流用可能

try:
    gme_revenue = get_quarterly_revenue_from_sec("0001326380", base_session=session)
    print(gme_revenue.tail())
except Exception as e:
    raise RuntimeError(f"Failed to fetch GME revenue via SEC API: {e}")


         Date     Revenue
12 2021-05-01  1276800000
13 2021-07-31  1183400000
14 2021-10-30  1296600000
15 2024-05-04   881800000
16 2025-05-03   732400000


In [14]:
# Q5: Tesla（株価 × Revenue）ダッシュボード
import plotly.graph_objects as go

def make_stock_revenue_fig(price_df: pd.DataFrame, revenue_df: pd.DataFrame, title: str):
    # price_df: DateTimeIndex, columnsに "Close" が含まれる想定
    # revenue_df: Date列, Revenue列
    price_m = price_df["Close"].resample("M").last()

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=price_m.index, y=price_m.values,
        mode="lines", name="Close Price"
    ))

    fig.add_trace(go.Bar(
        x=revenue_df["Date"], y=revenue_df["Revenue"],
        name="Quarterly Revenue", opacity=0.4, yaxis="y2"
    ))

    fig.update_layout(
        title=title,
        xaxis=dict(title="Date"),
        yaxis=dict(title="Close Price (USD)"),
        yaxis2=dict(title="Revenue (USD)", overlaying="y", side="right"),
        legend=dict(orientation="h"),
        template="plotly_white",
        hovermode="x unified",
        height=500,
    )
    return fig

fig_tsla = make_stock_revenue_fig(tsla_hist, tsla_revenue, "Tesla: Stock Price vs Quarterly Revenue")
fig_tsla.show()


In [15]:
# Q6: GameStop（株価 × Revenue）ダッシュボード
fig_gme = make_stock_revenue_fig(gme_hist, gme_revenue, "GameStop: Stock Price vs Quarterly Revenue")
fig_gme.show()


In [16]:
# Q7: バージョン確認 + データ概要
import sys, importlib, platform

def show_versions():
    print("Python:", sys.version)
    print("Platform:", platform.platform())
    for m in ["pandas", "yfinance", "requests", "plotly"]:
        try:
            mod = importlib.import_module(m)
            print(f"{m}:", getattr(mod, "__version__", "unknown"))
        except Exception as e:
            print(f"{m}: not installed ({e})")

show_versions()

summary = pd.DataFrame({
    "Dataset": ["TSLA Prices", "TSLA Revenue", "GME Prices", "GME Revenue"],
    "Rows": [len(tsla_hist), len(tsla_revenue), len(gme_hist), len(gme_revenue)],
    "Columns": [tsla_hist.shape[1], tsla_revenue.shape[1], gme_hist.shape[1], gme_revenue.shape[1]],
})
summary


Python: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:03:56) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-11-10.0.26100-SP0
pandas: 2.1.3
yfinance: 0.2.55
requests: 2.31.0
plotly: 6.0.1


,Dataset,Rows,Columns
0,TSLA Prices,2670,2
1,TSLA Revenue,12,2
2,GME Prices,3927,2
3,GME Revenue,17,2
